# Loading Library

In [112]:
# pandas
import pandas as pd

# numpy
import numpy as np

# defaultcit
from collections import defaultdict

# plot with folium
import folium
from IPython.core.display import HTML

# parsing time
from datetime import datetime, timedelta

# plotting with matplotlib
import matplotlib.pyplot as plt
import matplotlib
%matplotlib inline

# plotting with pylab
import pylab as P

# Clustering
from sklearn.cluster import MiniBatchKMeans, KMeans
import time
import json

# change prediction categories into labels
from sklearn import preprocessing
from sklearn import cross_validation 
from sklearn import svm
from sklearn.grid_search import GridSearchCV
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import re

from sklearn.naive_bayes import BernoulliNB
from sklearn.cross_validation import train_test_split
from sklearn.metrics import log_loss

from nltk import FreqDist

## Loading Training/Testing Data

** subset to only the fields we need and used in submission (Dates, DayOfWeek, PdDistrict, Address, X, Y, Category) **

In [540]:
test_df = pd.read_csv("test.csv")
train_df = pd.read_csv("train.csv")
#train_df = train_df.sample(n = 500000, random_state = 666)

In [216]:
train_df = pd.DataFrame(train_df)
test_df = pd.DataFrame(test_df)
# train_df = train_df.reset_index(drop=True)
train_df = train_df[['Dates','DayOfWeek','PdDistrict','Address','X','Y','Category']]
train_df.head()

,Dates,DayOfWeek,PdDistrict,Address,X,Y,Category
0,2015-05-13 23:53:00,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,WARRANTS
1,2015-05-13 23:53:00,Wednesday,NORTHERN,OAK ST / LAGUNA ST,-122.425892,37.774599,OTHER OFFENSES
2,2015-05-13 23:33:00,Wednesday,NORTHERN,VANNESS AV / GREENWICH ST,-122.424363,37.800414,OTHER OFFENSES
3,2015-05-13 23:30:00,Wednesday,NORTHERN,1500 Block of LOMBARD ST,-122.426995,37.800873,LARCENY/THEFT
4,2015-05-13 23:30:00,Wednesday,PARK,100 Block of BRODERICK ST,-122.438738,37.771541,LARCENY/THEFT


In [217]:
test_df.head()

,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412


In [218]:
#Look at the shape
print(train_df.shape)
print(test_df.shape)

(878049, 7)
(884262, 7)


## Feature Creation

** create feature for training data and also for test data **

** difference btw train/test is that training has category data **

In [541]:
def createFeature(train_df):
    #Getting Month of Year, Day of Month, Hour of Day, and Minute of Hour
    month_of_year = []
    day_of_month =[]
    hour_of_day =[]
    min_of_hour =[]
    month_and_day =[]
    for i in range(len(train_df.Dates.values)):
        moy = datetime.strptime(train_df.Dates.values[i], "%Y-%m-%d %H:%M:%S").month
        dom = datetime.strptime(train_df.Dates.values[i], "%Y-%m-%d %H:%M:%S").day
        hod = datetime.strptime(train_df.Dates.values[i], "%Y-%m-%d %H:%M:%S").hour
        moh = datetime.strptime(train_df.Dates.values[i], "%Y-%m-%d %H:%M:%S").minute
        month_of_year.append(moy)
        day_of_month.append(dom)
        hour_of_day.append(hod)
        min_of_hour.append(moh)
        #month_and_day.append((moy, dom))
    train_df['month_of_year'] = month_of_year #Month of the Year feature added
    train_df['day_of_month'] = day_of_month # Day of Month feature added
    train_df['hour_of_day'] = hour_of_day # Hour of Day feature added
    train_df['min_of_hour'] = min_of_hour # Minute of Hour Feature added
#    train_df['month_and_day'] = month_and_day # Month and Date
    
    #Creating Weeekday/Weekended Feature
    train_df['WeekdayWeeekend'] = train_df['DayOfWeek'].map( {'Monday': 'Weekday', 'Tuesday': 'Weekday', \
                                                          'Wednesday': 'Weekday', 'Thursday': 'Weekday', 'Friday': 'Weekend',\
                                                         'Saturday': 'Weekend', 'Sunday': 'Weekday'} ).astype(object)
    # Creating Midnight/Morning/Afternoon/Night Column
    train_df['TimeOfDay'] = train_df['hour_of_day'].map({0: 'Midnight', 1: 'Midnight', 2:'Midnight', 3:'Midnight', 4:'Morning', \
                                                     5:'Morning', 6:'Morning', 7:'Morning', 8:'Morning', 9:'Morning', \
                                                     10:'Morning', 11:'Morning', 12:'Afternoon', 13:'Afternoon', 14:'Afternoon', \
                                                     15:'Afternoon', 16:'Afternoon', 17:'Afternoon', 18:'Night', 19:'Night', \
                                                     20:'Night', 21:'Night', 22:'Midnight', 23:'Midnight'}).astype(object)
    # Creating Season Feature
    train_df['Season'] = train_df['month_of_year'].map({1: 'Winter', 2: 'Winter', 3:'Spring', 4:'Spring', 5:'Spring', \
                                                     6:'Spring', 7:'Summer', 8:'Summer', 9:'Summer', 10:'Autumn', \
                                                     11:'Autumn', 12:'Winter'}).astype(object)

    
    train_df['District_Type'] = train_df['PdDistrict'].map({'PARK': 'Other', 'CENTRAL': 'Other', 'MISSION': 'Corner', 'NORTHERN': 'Corner', 
                                                              'TENDERLOIN': 'Other', 'INGLESIDE': 'Street', 'TARAVAL': 'Street', 
                                                              'SOUTHERN': 'Other', 'BAYVIEW': 'Other', 'RICHMOND': 'Other'}).astype(object)

    ###Creating feature DANGEROUS_Address

    address_list = []
    for address in train_df['Address']:
        address_list.append(address)
    f = FreqDist(address_list)
    common_list = f.most_common(20)
    list_d = []
    for address in common_list:
        list_d.append(address[0])
    for address in train_df['Address'][:10000]:
        if address in list_d:
            d[address] = 'Danger'
        else:
            d[address] = 'A_Other'
    train_df['Address_Type'] = train_df['Address'].map(d)
    
    ###Creating feature DANGEROUS_DATES 
    date_list = []
    i = 0
    for date in train_df['Dates']:
        while i < len(train_df.Dates.values):
            moy = datetime.strptime(train_df.Dates.values[i], "%Y-%m-%d %H:%M:%S").month
            dom = datetime.strptime(train_df.Dates.values[i], "%Y-%m-%d %H:%M:%S").day
            date_list.append(((moy, dom)))
            i += 1
    train_df['Month_Day'] = date_list
    date_f = FreqDist(date_list)
    date_common_list = date_f.most_common(20)

    list_date = []
    dateDict = {}
    for date in date_common_list:
        list_date.append(date[0])

    j = 0
    for date in train_df['Dates']:
        while j < len(train_df.Dates.values):
            moy = datetime.strptime(train_df.Dates.values[j], "%Y-%m-%d %H:%M:%S").month
            dom = datetime.strptime(train_df.Dates.values[j], "%Y-%m-%d %H:%M:%S").day
            if (moy, dom) in list_date:
                dateDict[(moy, dom)] = 'Danger_Date'
            else:
                dateDict[(moy, dom)] = 'Not_Danger_Other'
            j += 1
    train_df['Date_Type'] = train_df['Month_Day'].map(dateDict)
    
    #Deleting features not needed
#     train_df = train_df.drop('Descript', 1)
#     train_df = train_df.drop('Resolution', 1)
    #train_df = train_df.drop('Address', 1)
    
    #Creating Dummy Variables
    WeekdayWeekend_dummies = pd.get_dummies(train_df.WeekdayWeeekend)
    TimeOfDay_dummies = pd.get_dummies(train_df.TimeOfDay)
    season_dummies = pd.get_dummies(train_df.Season)
    district_dummies = pd.get_dummies(train_df.PdDistrict)
    week_dummies = pd.get_dummies(train_df.DayOfWeek)
#     type_dummies = pd.get_dummies(train_df.Crime_Type)
    corner_dummies = pd.get_dummies(train_df.District_Type)
    address_dummies = pd.get_dummies(train_df.Address_Type)
    date_dummies = pd.get_dummies(train_df.Date_Type)
    
    train_df_new = pd.concat([train_df, WeekdayWeekend_dummies, TimeOfDay_dummies, season_dummies, district_dummies, week_dummies, corner_dummies, address_dummies, date_dummies], axis=1, join_axes=[train_df.index])
    print('Sanity Check')
    print(train_df.shape)
    print(WeekdayWeekend_dummies.shape)
    print(TimeOfDay_dummies.shape)
    print(season_dummies.shape)
    print(district_dummies.shape)
    print(week_dummies.shape)
#     print(type_dummies.shape)
    print(corner_dummies.shape)
    print(address_dummies.shape)
    print(date_dummies.shape)
    print(train_df_new.shape)
    print('Make sure the total adds up to the last number')
    
    return train_df_new
    
def createCrime(train_df):
    le_crime = preprocessing.LabelEncoder()
    crime = le_crime.fit_transform(train_df.Category)
    train_df['dummy_Category'] = crime
    
    return train_df

#Calling the feature creation function
train_df_new = createCrime(createFeature(train_df))
train_df_new.head()

Sanity Check
(878049, 20)
(878049, 2)
(878049, 4)
(878049, 4)
(878049, 10)
(878049, 7)
(878049, 3)
(878049, 2)
(878049, 2)
(878049, 54)
Make sure the total adds up to the last number


,Dates,Category,Descript,DayOfWeek,PdDistrict,Resolution,Address,X,Y,month_of_year,...,Tuesday,Wednesday,Corner,Other,Street,A_Other,Danger,Danger_Date,Not_Danger_Other,dummy_Category
0,2015-05-13 23:53:00,WARRANTS,WARRANT ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,5,...,0,1,1,0,0,1,0,0,1,37
1,2015-05-13 23:53:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",OAK ST / LAGUNA ST,-122.425892,37.774599,5,...,0,1,1,0,0,1,0,0,1,21
2,2015-05-13 23:33:00,OTHER OFFENSES,TRAFFIC VIOLATION ARREST,Wednesday,NORTHERN,"ARREST, BOOKED",VANNESS AV / GREENWICH ST,-122.424363,37.800414,5,...,0,1,1,0,0,1,0,0,1,21
3,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,NORTHERN,NONE,1500 Block of LOMBARD ST,-122.426995,37.800873,5,...,0,1,1,0,0,1,0,0,1,16
4,2015-05-13 23:30:00,LARCENY/THEFT,GRAND THEFT FROM LOCKED AUTO,Wednesday,PARK,NONE,100 Block of BRODERICK ST,-122.438738,37.771541,5,...,0,1,0,1,0,1,0,0,1,16


In [220]:
test_df_new = createFeature(test_df)
test_df_new.head()

Sanity Check
(884262, 16)
(884262, 2)
(884262, 4)
(884262, 4)
(884262, 10)
(884262, 7)
(884262, 3)
(884262, 2)
(884262, 48)
Make sure the total adds up to the last number


,Id,Dates,DayOfWeek,PdDistrict,Address,X,Y,month_of_year,day_of_month,hour_of_day,...,Saturday,Sunday,Thursday,Tuesday,Wednesday,Corner,Other,Street,A_Other,Danger
0,0,2015-05-10 23:59:00,Sunday,BAYVIEW,2000 Block of THOMAS AV,-122.399588,37.735051,5,10,23,...,0,1,0,0,0,0,1,0,1,0
1,1,2015-05-10 23:51:00,Sunday,BAYVIEW,3RD ST / REVERE AV,-122.391523,37.732432,5,10,23,...,0,1,0,0,0,0,1,0,1,0
2,2,2015-05-10 23:50:00,Sunday,NORTHERN,2000 Block of GOUGH ST,-122.426002,37.792212,5,10,23,...,0,1,0,0,0,1,0,0,1,0
3,3,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,5,10,23,...,0,1,0,0,0,0,0,1,1,0
4,4,2015-05-10 23:45:00,Sunday,INGLESIDE,4700 Block of MISSION ST,-122.437394,37.721412,5,10,23,...,0,1,0,0,0,0,0,1,1,0


In [221]:
test_df_new.shape, train_df_new.shape

((884262, 48), (878049, 49))

##Prediction

### Feature Inclusion/Exclusion

In [542]:
# Lat/Long
X_feature = (True, 'X')
Y_feature = (True, 'Y')

# Time
Month_feature = (True, 'month_of_year')
Day_feature = (True, 'day_of_month')
Hour_feature = (True, 'hour_of_day')
Min_feature = (True, 'min_of_hour')

# Day of Week
Friday_feature = (True, 'Friday') 
Monday_feature = (True, 'Monday')
Saturday_feature = (True, 'Saturday')
Sunday_feature = (True, 'Sunday')
Thursday_feature = (True, 'Thursday')
Tuesday_feature = (True, 'Tuesday')
Wednesday_feature = (True, 'Wednesday')

#Weekday/Weekend
Weekday_feature = (True, 'Weekday') 
Weekend_feature = (True, 'Weekend') 

#Season 
Autumn_feature = (True, 'Autumn') 
Spring_feature = (True, 'Spring') 
Summer_feature = (True, 'Summer') 
Winter_feature = (True, 'Winter') 

#Time of Day
Midnight_feature = (True, 'Midnight') 
Morning_feature = (True, 'Morning') 
Afternoon_feature = (True, 'Afternoon') 
Night_feature = (True, 'Night') 

# District
BAYV_feature = (True, 'BAYVIEW')
CENT_feature = (True, 'CENTRAL')
INGL_feature = (True, 'INGLESIDE')
MISS_feature = (True, 'MISSION')
NORT_feature = (True, 'NORTHERN')
PARK_feature = (True, 'PARK')
RICH_feature = (True, 'RICHMOND')
SOUT_feature = (True, 'SOUTHERN')
TARA_feature = (True, 'TARAVAL')
TEND_feature = (True, 'TENDERLOIN')

# # Crime Type
# Blue_feature = (False, 'Blue')
# White_feature = (False, 'White')
# Other_feature = (False, 'Other')

# District Type
Corner_feature = (True, 'Corner')
Street_feature = (True, 'Street')
District_Other_feature = (True, 'Other')

# Address Type
Address_Danger_feature = (True, 'Danger')
Address_Other_feature = (True, 'A_Other')

# Date Type
Danger_Date_feature = (True, 'Danger_Date')
Not_Danger_Date_feature = (True, 'Not_Danger_Other')

In [580]:
feature_list = [
                X_feature, Y_feature,
                Month_feature, Day_feature, Hour_feature, Min_feature,
                #Monday_feature, Tuesday_feature, Wednesday_feature, Thursday_feature, Friday_feature, 
                Saturday_feature, Sunday_feature,
                Weekday_feature, Weekend_feature, 
                #Autumn_feature, Spring_feature, Summer_feature, Winter_feature,
                Midnight_feature, Morning_feature, Afternoon_feature, Night_feature, 
                BAYV_feature, CENT_feature, INGL_feature, MISS_feature, NORT_feature, PARK_feature, 
                RICH_feature, SOUT_feature, TARA_feature, TEND_feature,
                #Corner_feature, Street_feature, District_Other_feature,
                Address_Danger_feature, Address_Other_feature,
                Danger_Date_feature, Not_Danger_Date_feature
               ]
features = [str(x[1]) for x in feature_list if x[0]]

# Trainding and Validation

In [586]:
training, validation = cross_validation.train_test_split(train_df_new, train_size=.60)
#2.55820 current score
model = BernoulliNB()
model.fit(training[features], training['dummy_Category'])
predicted = np.array(model.predict_proba(validation[features]))
log_loss(validation['dummy_Category'], predicted)

2.551351505727157

### Logistic Regression

In [16]:
#Logistic Regression for comparison
model = LogisticRegression(C=.1)
model.fit(training[features], training['dummy_Category'])
predicted = np.array(model.predict_proba(validation[features]))
log_loss(validation['dummy_Category'], predicted) 

ValueError: y_true and y_pred have different number of classes 38, 39

In [ ]:
param_grid = {'C': [0.150, 0.125, 0.1, .075, .05] }

param_scores = ['precision', 'recall']

Haven't got the gridsearch on logistic regression working yet (possibly need to use less than full data

##Submission with Bernoulli

In [226]:
test_df_new = test_df_new.ix[:,1:len(test_df_new)]
test_df_new.shape

(884262, 47)

In [227]:
model = BernoulliNB()
model.fit(train_df_new[features], train_df_new['dummy_Category'])

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [228]:
predicted = np.array(model.predict_proba(test_df_new[features]))

In [232]:
ex_submission = pd.read_csv("sampleSubmission.csv")

In [233]:
result= pd.DataFrame(predicted, columns=ex_submission.columns[1:len(ex_submission)])
result.to_csv('testResult.csv', index = True, index_label = 'Id' )

## KNN

In [91]:
knn = KNeighborsClassifier(n_neighbors=50, weights='uniform')
knn.fit(training[features], training['dummy_Category']) 
knn_predicted = np.array(knn.predict_proba(validation[features]))
log_loss(validation['dummy_Category'], knn_predicted)

8.5881297183958729